In [27]:
import pandas as pd
import os
import spacy_annotator as spa
import spacy
import ast

In [6]:
os.curdir

'.'

In [9]:
df_train = pd.read_csv("/Users/senghok/Documents/Internship 2A/code/data/train.csv",sep =',')
df_test = pd.read_csv("/Users/senghok/Documents/Internship 2A/code/data/test.csv", sep = ',')
df_valid = pd.read_csv("/Users/senghok/Documents/Internship 2A/code/data/valid.csv", sep = ',')

In [10]:
df_train.head()

,sentence,labels
0,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
1,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,The party is divided over Britain 's participa...,"O,O,O,O,O,B-gpe,O,O,O,O,B-geo,O,O,O,O,O,O,O,B-..."
3,Two Germans and four Nigerian oil workers were...,"O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,The German firm works as a sub-contractor for ...,"O,B-gpe,O,O,O,O,O,O,B-org,O"


In [21]:

def spacy_format(row):
    tokens = row["sentence"].split(" ")
    labels = row["labels"].split(",")
    entities = []
    
    current_pos = 0
    for token, label in zip(tokens, labels):
        # skip leading whitespace to find token start
        start = row["sentence"].find(token, current_pos)
        end = start + len(token)
        current_pos = end  # update pointer for next search
        
        if label != "O":
            entities.append((start, end, label))
    
    return (row["sentence"], {"entities": entities})


In [13]:
df_train["annotation"] = df_train.apply(spacy_format, axis=1)

In [55]:
df_valid["annotation"] = df_valid.apply(spacy_format, axis = 1)

In [19]:
df_train["labels"]

0        O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,...
1                            O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
2        O,O,O,O,O,B-gpe,O,O,O,O,B-geo,O,O,O,O,O,O,O,B-...
3        O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
4                              O,B-gpe,O,O,O,O,O,O,B-org,O
                               ...                        
34274    B-gpe,O,O,O,O,O,O,B-gpe,O,O,O,O,O,O,O,O,O,B-ge...
34275    B-gpe,O,O,O,O,O,O,O,B-tim,O,O,O,O,O,O,O,O,O,O,...
34276                                O,O,O,O,O,O,O,O,O,O,O
34277                                O,O,O,O,O,O,O,O,O,O,O
34278                                  B-gpe,O,O,O,O,O,O,O
Name: labels, Length: 34279, dtype: object

In [18]:
df_train["annotation"][2]

("The party is divided over Britain 's participation in the Iraq conflict and the continued deployment of 8,500 British troops in that country .",
 {'entities': [(26, 33, 'B-gpe'), (58, 62, 'B-geo'), (110, 117, 'B-gpe')]})

In [25]:
df_train.drop(columns = ["labels"],inplace= True)

In [56]:
df_valid.drop(columns = ["labels"],inplace= True)

In [37]:
df_train["annotation"][0]

('Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
 {'entities': [(109, 113, 'B-per')]})

In [36]:
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.lang.en import English

nlp = English()  # or load the language model you're using
doc_bin = DocBin()


In [58]:
def convert_to_spacy(data):
    for text, annotations in data["annotation"]:
        doc = nlp.make_doc(text)
        entities = annotations.get("entities", [])
        ents = []
        for start, end, label in entities:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents
        doc_bin.add(doc)
    return doc_bin


In [59]:
doc_bin = convert_to_spacy(df_valid)


In [61]:
doc_bin.to_disk("valid.spacy")

In [49]:
docs = list(doc_bin.get_docs(nlp.vocab))